In [1]:
"""
Please run notebook locally (if you have all the dependencies and a GPU). 
Technically you can run this notebook on Google Colab but you need to set up microphone for Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Set up microphone for Colab


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg portaudio19-dev
!pip install text-unidecode
!pip install pyaudio

# ## Install NeMo
BRANCH = 'r2.0.0rc0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Grab the config we'll use in this example
!mkdir configs

zsh:1: command not found: apt-get
zsh:1: no matches found: git+https://github.com/NVIDIA/NeMo.git@r2.0.0rc0#egg=nemo_toolkit[asr]


This notebook demonstrates automatic speech recognition (ASR) from a microphone's stream in NeMo.

It is **not a recommended** way to do inference in production workflows. And the incompatibility of components could lead to failure of running this notebook locally with container, we might deprecate this notebook and provide a better tutorial in soon releases. If you are interested in production-level inference using NeMo ASR models, please refer to NVIDIA RIVA: https://developer.nvidia.com/riva

The notebook requires PyAudio library to get a signal from an audio device.
For Ubuntu, please run the following commands to install it:
```
sudo apt install python3-pyaudio
pip install pyaudio
```

In [22]:
import numpy as np
import pyaudio as pa
import os, time

In [23]:
import nemo
import nemo.collections.asr as nemo_asr

In [24]:
# sample rate, Hz
SAMPLE_RATE = 16000

## Restore the model from NGC

In [25]:
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained('QuartzNet15x5Base-En')

[NeMo I 2024-09-06 08:38:59 nemo_logging:381] Found existing object /Users/manojdannana/.cache/torch/NeMo/NeMo_1.21.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2024-09-06 08:38:59 nemo_logging:381] Re-using file from: /Users/manojdannana/.cache/torch/NeMo/NeMo_1.21.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2024-09-06 08:38:59 nemo_logging:381] Instantiating model from pre-trained checkpoint
[NeMo I 2024-09-06 08:39:00 nemo_logging:381] PADDING: 16
[NeMo I 2024-09-06 08:39:00 nemo_logging:381] Model EncDecCTCModel was successfully restored from /Users/manojdannana/.cache/torch/NeMo/NeMo_1.21.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


## Observing the config of the model

In [26]:
from omegaconf import OmegaConf
import copy

In [27]:
# Preserve a copy of the full config
cfg = copy.deepcopy(asr_model._cfg)
print(OmegaConf.to_yaml(cfg))

preprocessor:
  _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
  normalize: per_feature
  window_size: 0.02
  sample_rate: 16000
  window_stride: 0.01
  window: hann
  features: 64
  n_fft: 512
  frame_splicing: 1
  dither: 1.0e-05
  stft_conv: false
spec_augment:
  _target_: nemo.collections.asr.modules.SpectrogramAugmentation
  rect_freq: 50
  rect_masks: 5
  rect_time: 120
encoder:
  _target_: nemo.collections.asr.modules.ConvASREncoder
  feat_in: 64
  activation: relu
  conv_mask: true
  jasper:
  - filters: 256
    repeat: 1
    kernel:
    - 33
    stride:
    - 2
    dilation:
    - 1
    dropout: 0.0
    residual: false
    separable: true
  - filters: 256
    repeat: 5
    kernel:
    - 33
    stride:
    - 1
    dilation:
    - 1
    dropout: 0.0
    residual: true
    separable: true
  - filters: 256
    repeat: 5
    kernel:
    - 33
    stride:
    - 1
    dilation:
    - 1
    dropout: 0.0
    residual: true
    separable: true
  - filters: 256
 

### Modify preprocessor parameters for inference

In [28]:
# Make config overwrite-able
OmegaConf.set_struct(cfg.preprocessor, False)

# some changes for streaming scenario
cfg.preprocessor.dither = 0.0
cfg.preprocessor.pad_to = 0

# spectrogram normalization constants
normalization = {}
normalization['fixed_mean'] = [
     -14.95827016, -12.71798736, -11.76067913, -10.83311182,
     -10.6746914,  -10.15163465, -10.05378331, -9.53918999,
     -9.41858904,  -9.23382904,  -9.46470918,  -9.56037,
     -9.57434245,  -9.47498732,  -9.7635205,   -10.08113074,
     -10.05454561, -9.81112681,  -9.68673603,  -9.83652977,
     -9.90046248,  -9.85404766,  -9.92560366,  -9.95440354,
     -10.17162966, -9.90102482,  -9.47471025,  -9.54416855,
     -10.07109475, -9.98249912,  -9.74359465,  -9.55632283,
     -9.23399915,  -9.36487649,  -9.81791084,  -9.56799225,
     -9.70630899,  -9.85148006,  -9.8594418,   -10.01378735,
     -9.98505315,  -9.62016094,  -10.342285,   -10.41070709,
     -10.10687659, -10.14536695, -10.30828702, -10.23542833,
     -10.88546868, -11.31723646, -11.46087382, -11.54877829,
     -11.62400934, -11.92190509, -12.14063815, -11.65130117,
     -11.58308531, -12.22214663, -12.42927197, -12.58039805,
     -13.10098969, -13.14345864, -13.31835645, -14.47345634]
normalization['fixed_std'] = [
     3.81402054, 4.12647781, 4.05007065, 3.87790987,
     3.74721178, 3.68377423, 3.69344,    3.54001005,
     3.59530412, 3.63752368, 3.62826417, 3.56488469,
     3.53740577, 3.68313898, 3.67138151, 3.55707266,
     3.54919572, 3.55721289, 3.56723346, 3.46029304,
     3.44119672, 3.49030548, 3.39328435, 3.28244406,
     3.28001423, 3.26744937, 3.46692348, 3.35378948,
     2.96330901, 2.97663111, 3.04575148, 2.89717604,
     2.95659301, 2.90181116, 2.7111687,  2.93041291,
     2.86647897, 2.73473181, 2.71495654, 2.75543763,
     2.79174615, 2.96076456, 2.57376336, 2.68789782,
     2.90930817, 2.90412004, 2.76187531, 2.89905006,
     2.65896173, 2.81032176, 2.87769857, 2.84665271,
     2.80863137, 2.80707634, 2.83752184, 3.01914511,
     2.92046439, 2.78461139, 2.90034605, 2.94599508,
     2.99099718, 3.0167554,  3.04649716, 2.94116777]

cfg.preprocessor.normalize = normalization

# Disable config overwriting
OmegaConf.set_struct(cfg.preprocessor, True)

## Setup preprocessor with these settings

In [29]:
asr_model.preprocessor = asr_model.from_config_dict(cfg.preprocessor)

[NeMo I 2024-09-06 08:39:12 nemo_logging:381] PADDING: 0


In [30]:
# Set model to inference mode
asr_model.eval();

In [31]:
asr_model = asr_model.to(asr_model.device)

## Setting up data for Streaming Inference

In [32]:
from nemo.core.classes import IterableDataset
from nemo.core.neural_types import NeuralType, AudioSignal, LengthsType
import torch
from torch.utils.data import DataLoader

In [33]:
# simple data layer to pass audio signal
class AudioDataLayer(IterableDataset):
    @property
    def output_types(self):
        return {
            'audio_signal': NeuralType(('B', 'T'), AudioSignal(freq=self._sample_rate)),
            'a_sig_length': NeuralType(tuple('B'), LengthsType()),
        }

    def __init__(self, sample_rate):
        super().__init__()
        self._sample_rate = sample_rate
        self.output = True
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if not self.output:
            raise StopIteration
        self.output = False
        return torch.as_tensor(self.signal, dtype=torch.float32), \
               torch.as_tensor(self.signal_shape, dtype=torch.int64)
        
    def set_signal(self, signal):
        self.signal = signal.astype(np.float32)/32768.
        self.signal_shape = self.signal.size
        self.output = True

    def __len__(self):
        return 1

In [34]:
data_layer = AudioDataLayer(sample_rate=cfg.preprocessor.sample_rate)
data_loader = DataLoader(data_layer, batch_size=1, collate_fn=data_layer.collate_fn)

In [35]:
# inference method for audio signal (single instance)
def infer_signal(model, signal):
    data_layer.set_signal(signal)
    batch = next(iter(data_loader))
    audio_signal, audio_signal_len = batch
    audio_signal, audio_signal_len = audio_signal.to(asr_model.device), audio_signal_len.to(asr_model.device)
    log_probs, encoded_len, predictions = model.forward(
        input_signal=audio_signal, input_signal_length=audio_signal_len
    )
    return log_probs

In [66]:
# class for streaming frame-based ASR
# 1) use reset() method to reset FrameASR's state
# 2) call transcribe(frame) to do ASR on
#    contiguous signal's frames
class FrameASR:
    
    def __init__(self, model_definition,
                 frame_len=2, frame_overlap=2.5, 
                 offset=10):
        '''
        Args:
          frame_len: frame's duration, seconds
          frame_overlap: duration of overlaps before and after current frame, seconds
          offset: number of symbols to drop for smooth streaming
        '''
        # self.vocab = list(model_definition['labels'])
        # self.vocab.append('_')
        self.vocab = ['_'] + list(model_definition['labels'])
        
        self.sr = model_definition['sample_rate']
        self.frame_len = frame_len
        self.n_frame_len = int(frame_len * self.sr)
        self.frame_overlap = frame_overlap
        self.n_frame_overlap = int(frame_overlap * self.sr)
        timestep_duration = model_definition['AudioToMelSpectrogramPreprocessor']['window_stride']
        for block in model_definition['JasperEncoder']['jasper']:
            timestep_duration *= block['stride'][0] ** block['repeat']
        self.n_timesteps_overlap = int(frame_overlap / timestep_duration) - 2
        self.buffer = np.zeros(shape=2*self.n_frame_overlap + self.n_frame_len,
                               dtype=np.float32)
        self.offset = offset
        self.reset()
        
    def _decode(self, frame, offset=0):
        assert len(frame)==self.n_frame_len
        print(f"Buffer before shift: {self.buffer}")
        self.buffer[:-self.n_frame_len] = self.buffer[self.n_frame_len:]
        self.buffer[-self.n_frame_len:] = frame
        print(f"Buffer after shift: {self.buffer}")
        logits = infer_signal(asr_model, self.buffer).cpu().numpy()[0]
        print(f"Logits (first 5 steps): {logits[:5]}")
        # print(logits.shape)
        decoded = self._greedy_decoder(
            logits[self.n_timesteps_overlap:-self.n_timesteps_overlap], 
            self.vocab
        )
        return decoded[:len(decoded)-offset]
    
    @torch.no_grad()
    def transcribe(self, frame=None, merge=True):
        if frame is None:
            frame = np.zeros(shape=self.n_frame_len, dtype=np.float32)
        if len(frame) < self.n_frame_len:
            frame = np.pad(frame, [0, self.n_frame_len - len(frame)], 'constant')
        unmerged = self._decode(frame, self.offset)
        if not merge:
            return unmerged
        return self.greedy_merge(unmerged)
    
    def reset(self):
        '''
        Reset frame_history and decoder's state
        '''
        self.buffer=np.zeros(shape=self.buffer.shape, dtype=np.float32)
        self.prev_char = ''

    @staticmethod
    def _greedy_decoder(logits, vocab):
        s = ''
        for i in range(logits.shape[0]):
            print(f"Timestep {i}: {vocab[np.argmax(logits[i])]}")
            s += vocab[np.argmax(logits[i])]
        return s

    def greedy_merge(self, s):
        s_merged = ''
        
        for i in range(len(s)):
            if s[i] != self.prev_char:
                self.prev_char = s[i]
                if self.prev_char != '_':
                    s_merged += self.prev_char
        return s_merged


# Streaming Inference

Streaming inference depends on a few factors, such as the frame length and buffer size. Experiment with a few values to see their effects in the below cells.

In [67]:
# duration of signal frame, seconds
FRAME_LEN = 1.0
# number of audio channels (expect mono signal)
CHANNELS = 1

CHUNK_SIZE = int(FRAME_LEN*SAMPLE_RATE)
asr = FrameASR(model_definition = {
                   'sample_rate': SAMPLE_RATE,
                   'AudioToMelSpectrogramPreprocessor': cfg.preprocessor,
                   'JasperEncoder': cfg.encoder,
                   'labels': cfg.decoder.vocabulary
               },
               frame_len=FRAME_LEN, frame_overlap=2, 
               offset=4)

In [68]:
asr.reset()

p = pa.PyAudio()
print('Available audio input devices:')
input_devices = []
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    if dev.get('maxInputChannels'):
        input_devices.append(i)
        print(i, dev.get('name'))

if len(input_devices):
    dev_idx = -2
    while dev_idx not in input_devices:
        print('Please type input device ID:')
        dev_idx = int(input())

    empty_counter = 0

    def callback(in_data, frame_count, time_info, status):
        global empty_counter
        signal = np.frombuffer(in_data, dtype=np.int16)
        signal = signal / np.max(np.abs(signal))
        
        # Detailed print for debugging
        print(f"Received signal data: {signal[:10]}... Length: {len(signal)}")
        
        text = asr.transcribe(signal)
        
        print(f"Text: {text}")
        
        if len(text):
            print(text, end='')
            empty_counter = asr.offset
        elif empty_counter > 0:
            empty_counter -= 1
            if empty_counter == 0:
                print(' ', end='')
        
        return (in_data, pa.paContinue)



    stream = p.open(format=pa.paInt16,
                channels=CHANNELS,
                rate=SAMPLE_RATE,
                input=True,
                input_device_index=dev_idx,
                stream_callback=callback,
                frames_per_buffer=CHUNK_SIZE)

    print('Listening...')

    stream.start_stream()
    
    # Interrupt kernel and then speak for a few more words to exit the pyaudio loop !
    try:
        while stream.is_active():
            time.sleep(0.1)
    finally:        
        stream.stop_stream()
        stream.close()
        p.terminate()

        print()
        print("PyAudio stopped")
    
else:
    print('ERROR: No audio input device found.')

Available audio input devices:
0 Manoj’s iPhone Microphone
1 MacBook Air Microphone
3 Microsoft Teams Audio
Please type input device ID:


 0


Listening...
Received signal data: [-0.17241379 -0.34482759 -0.20689655 -0.24137931 -0.31034483 -0.24137931
 -0.13793103 -0.13793103 -0.17241379 -0.17241379]... Length: 16000
Buffer before shift: [0. 0. 0. ... 0. 0. 0.]
Buffer after shift: [0.         0.         0.         ... 0.03448276 0.10344828 0.06896552]
Logits (first 5 steps): [[-4.1564174  -5.158637   -5.088897   -5.28916    -5.1556253  -4.6792035
  -5.47671    -5.6465383  -4.6369896  -5.0782523  -6.071766   -5.605948
  -5.396924   -5.2040653  -5.1495137  -5.098709   -5.2204223  -6.6605186
  -4.8056583  -4.801425   -4.2214227  -5.2536674  -5.717507   -5.2386785
  -6.5733867  -5.209139   -6.567171   -6.1580877  -0.1757779 ]
 [-5.9227033  -5.836525   -6.0511003  -6.0877075  -6.210748   -5.4067335
  -6.1263547  -6.71648    -5.687315   -5.800773   -7.07845    -6.5669765
  -6.172377   -6.026078   -5.9479346  -5.9118075  -5.9700837  -7.6671166
  -5.608802   -5.4761195  -5.025568   -5.973077   -6.552486   -6.0364523
  -7.4872236  -5.9

[NeMo W 2024-09-06 11:24:33 nemo_logging:393] /var/folders/0d/y2bmjpkj2k95_b28klgd5hzw0000gn/T/ipykernel_5963/4064267191.py:23: RuntimeWarning: invalid value encountered in divide
      signal = signal / np.max(np.abs(signal))
    


Received signal data: [nan nan nan nan nan nan nan nan nan nan]... Length: 16000
Buffer before shift: [0. 0. 0. ... 0. 0. 0.]
Buffer after shift: [ 0.  0.  0. ... nan nan nan]
Logits (first 5 steps): [[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan]]
Timestep 0: _
Timestep 1: _
Timestep 2: _
Timestep 3: _
Timestep 4: _
Timestep 5: _
Timestep 6: _
Timestep 7: _
Timestep 8: _
Timestep 9: _
Timestep 10: _
Timestep 11: _
Timestep 12: _
Timestep 

KeyboardInterrupt: 

## Result
#### Having issues while transcribing, the input audio signal is being recorded normally without any issues, logit normalization is also good, I am suspecting the issue might be with greedy decoder or some other model parameter

# Saving as temp audio file and trascribe after

In [64]:
import nemo.collections.asr as nemo_asr
import sounddevice as sd
import numpy as np
import soundfile as sf
import os
import torch

# Load pre-trained ASR model
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

# Function to record and transcribe audio
def record_and_transcribe(duration=5, sample_rate=16000, temp_wav_file='temp_audio.wav'):
    print("Recording audio...")
    
    # Record audio from the microphone for the given duration
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Wait until the recording is finished
    
    # Convert audio to 1D numpy array
    audio_data = np.squeeze(audio_data)

    # Save recorded audio to a temporary wav file
    sf.write(temp_wav_file, audio_data, sample_rate)

    print("Transcribing...")

    # Transcribe the audio from the saved .wav file
    with torch.no_grad():
        transcription = asr_model.transcribe([temp_wav_file])
    
    print("Transcription: ", transcription[0])

    # Remove the temporary wav file
    os.remove(temp_wav_file)

record_and_transcribe(duration=50)


[NeMo I 2024-09-06 11:13:14 nemo_logging:381] Found existing object /Users/manojdannana/.cache/torch/NeMo/NeMo_1.21.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2024-09-06 11:13:14 nemo_logging:381] Re-using file from: /Users/manojdannana/.cache/torch/NeMo/NeMo_1.21.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2024-09-06 11:13:14 nemo_logging:381] Instantiating model from pre-trained checkpoint
[NeMo I 2024-09-06 11:13:14 nemo_logging:381] PADDING: 16
[NeMo I 2024-09-06 11:13:15 nemo_logging:381] Model EncDecCTCModel was successfully restored from /Users/manojdannana/.cache/torch/NeMo/NeMo_1.21.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
Recording audio...
Transcribing...


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcription:  hi hello howbout you i hope you'r ren and i will thank you good morning good night


## Result
#### It transcribes pretty well but has issues when the word pronounciation isn't accurate